In [2]:
import pandas as pd
import numpy as np
import re

## Cleaning

In [3]:
df_ds= pd.read_csv('Data_Scientist.csv')
def clean(df):
    for i in range(len(df)):
        df.loc[i,"Description"]=re.sub("[\n]"," ",df.loc[i,"Description"])
    return df
df_desc_clean=clean(df_ds)

## Translating

In [65]:
import requests, uuid, json
import pandas as pd
import time

# Add your key and endpoint
key = ""
endpoint = "https://api.cognitive.microsofttranslator.com"
# location, also known as regio
location = "westeurope"

# Define the path and constructed URL for the Translator Text API
path = '/detect'
detect_url = endpoint + path

# Define the parameters for the Detect method
detect_params = {
    'api-version': '3.0',
}

detect_headers = {
    'Ocp-Apim-Subscription-Key': key,
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# Define the path and constructed URL for the Translate method
translate_path = '/translate'
translate_url = endpoint + translate_path

# Define the parameters for the Translate method
translate_params = {
    'api-version': '3.0',
    'to': 'en'
}

translate_headers = {
    'Ocp-Apim-Subscription-Key': key,
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}
def translate_text(text):
    
    # Call the Detect method to determine the language of the text
    detect_body = [{
        'text': text
    }]
    
    detect_response = requests.post(detect_url, params=detect_params, headers=detect_headers, json=detect_body)
    print(detect_response)
    detect_result = detect_response.json()[0]
    detect_language = detect_result['language']
    
    # Call the Translate method to translate the text into English
    translate_params['from'] = detect_language
    translate_body = [{
        'text': text
    }]
    try:
        translate_response = requests.post(translate_url, params=translate_params, headers=translate_headers, json=translate_body)
        translate_result = translate_response.json()[0]
        translated_text = translate_result["translations"][0]["text"]#translate_result['text']

        return True,translated_text
    except:
        print("Nieudane")
        return False, "Nieudane"
    
# country USA or Region england out of translating (worthless)    
# Call the translate_text function for each row in df_desc_clean
rows_translated = []
i=0
num=0
#I encountered a problem compiling it, errros were popping up, but surprisingly when compiling small chunks 
#it didn't break down where it had previously broken down
#for index, row in df_desc_clean.loc[(df_desc_clean["Country"] != ["USA","Wielka Brytania","Polska"]) & (df_desc_clean["Region"] != "Anglia")].iterrows():
for index, row in df_desc_clean[~df_desc_clean["Country"].isin(["USA","Wielka Brytania","Polska"])].iterrows():
    print(i,row["Country"])
    #break
    i+=1
    if i >130:
        state, translated_text = translate_text(row['Description'])
        if state:
            df_desc_clean.loc[index,"Description"] = translated_text
        else:
            num+=1
print(num)


0 Niemcy
1 Niemcy
2 Niemcy
3 Niemcy
4 Niemcy
5 Niemcy
6 Niemcy
7 Niemcy
8 Niemcy
9 Niemcy
10 Niemcy
11 Niemcy
12 Niemcy
13 Niemcy
14 Niemcy
15 Niemcy
16 Niemcy
17 Niemcy
18 Niemcy
19 Niemcy
20 Niemcy
21 Niemcy
22 Austria
23 Niemcy
24 Niemcy
25 Niemcy
26 Niemcy
27 Niemcy
28 Niemcy
29 Niemcy
30 Niemcy
31 Niemcy
32 Austria
33 Niemcy
34 Niemcy
35 Niemcy
36 Niemcy
37 Niemcy
38 Niemcy
39 Niemcy
40 Niemcy
41 Niemcy
42 Austria
43 Szwajcaria
44 Szwajcaria
45 Niemcy
46 Niemcy
47 Niemcy
48 Niemcy
49 Niemcy
50 Szwajcaria
51 Szwajcaria
52 Niemcy
53 Niemcy
54 Niemcy
55 Austria
56 Szwajcaria
57 Szwajcaria
58 Niemcy
59 Niemcy
60 Niemcy
61 Szwajcaria
62 Niemcy
63 Niemcy
64 Niemcy
65 Niemcy
66 Szwajcaria
67 Niemcy
68 Niemcy
69 Niemcy
70 Niemcy
71 Niemcy
72 Niemcy
73 Austria
74 Niemcy
75 Niemcy
76 Niemcy
77 Niemcy
78 Austria
79 Niemcy
80 Niemcy
81 Niemcy
82 Niemcy
83 Niemcy
84 Niemcy
85 Austria
86 Niemcy
87 Niemcy
88 Austria
89 Niemcy
90 Niemcy
91 Szwajcaria
92 Niemcy
93 Niemcy
94 Szwajcaria
95 Niemcy
96

In [67]:
df_desc_clean.to_csv("translated_ds.csv",index=False)

Some of translations have german/polish begining, However Those aren't important